In [1]:
import pandas as pd
import numpy as np

import os
import os.path
import re
import string
import collections
import math
from collections import Counter
import random

frequentwords = ['a', 'all', 'also', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', 'do', 'down', 'even', 'every', 'for', 'from', 'had', 'has', 'have', 'her', 'his', 'if', 'in', 'into', 'is', 'it', 'its', 'may', 'more', 'must', 'my', 'no', 'not', 'now', 'of', 'on', 'one', 'only', 'or', 'our', 'shall', 'should', 'so', 'some', 'such', 'than', 'that', 'the', 'their', 'then', 'there', 'things', 'this', 'to', 'up', 'upon', 'was', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'would', 'your']

path = '../data/Raw/blogs/blogs/'
files = os.listdir(path)
files.sort() 

df = pd.read_csv('../data/Processed/dataframe_2.csv')
df = df.sort_values('total_words', ascending = False)

In [2]:
def get_train_test_sample(df,train_n,test_n,take):
    target = train_n+test_n
    temp = df[df['total_words'] >= target].head(take)
    train_vecs = [] 
    test_vecs = [] 
    ids = []
    for i in temp.iterrows():
        words = i[1][-1] - i[1][-2]
 
        random_words =random.sample([i for i in range(0,words)],train_n)

        trvec = [] 
        prev_len = 0
        for x in i[1].index:
            if np.isin(x,frequentwords) == True:
                size = i[1][x]
                count = [i for i in random_words if prev_len <= i < prev_len + size ]
                prev_len = size
                trvec.append(len(count))

        random_words = random.sample(list(range(0,words)),test_n)

        tevec = [] 
        prev_len = 0
        for x in i[1].index:
            if np.isin(x,frequentwords) == True:
                size = i[1][x]
                count = [i for i in random_words if prev_len <= i < prev_len + size ]
                prev_len = size
                tevec.append(len(count))

        ids.append(i[1][0])
        test_vecs.append(tevec)
        train_vecs.append(trvec)
    return([ids,test_vecs,train_vecs])

In [14]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

trains = []
tests = []
ys = []
for i in [100,200,250,500,750,1000,2000,3000,4000]:
    ids,test,train = get_train_test_sample(df,1000,int(1000/5),i)
    X_test = pd.DataFrame(test)
    X_train = pd.DataFrame(train)
    y = ids

    trains.append(X_train)
    tests.append(X_test)
    ys.append(y)


In [15]:
tree_predictions = []
knn_predictions = []
net_predictions = []
log_predictions = []
no_scaler_t = []

for j in range (0,len(trains)):
    print(i)
    X_train = trains[j]
    X_test = tests[j]
    y = ys[j]

    tree_model = DecisionTreeClassifier(max_depth = 200)
    tree_model.fit(X_train,y)
    t_predictions = tree_model.predict(X_test)
    no_scaler_t.append(sum(np.where(y-t_predictions == 0,1,0))/len(t_predictions))

    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)

    tree_model = DecisionTreeClassifier(max_depth = 200)
    knn_model = KNeighborsClassifier(algorithm = 'brute')
    net_model = MLPClassifier(solver='adam', alpha=0.001,hidden_layer_sizes=(100, 70),activation = 'logistic',max_iter = 1000)
    log_model = LogisticRegression(solver = 'saga',penalty = 'l1',max_iter = 1000)

    tree_model.fit(X_train,y)
    t_predictions = tree_model.predict(X_test)
    tree_predictions.append(sum(np.where(y-t_predictions == 0,1,0))/len(t_predictions))

    knn_model.fit(X_train,y)
    k_predictions = knn_model.predict(X_test)
    knn_predictions.append(sum(np.where(y-k_predictions == 0,1,0))/len(k_predictions))


    net_model.fit(X_train,y)
    n_predictions = net_model.predict(X_test)
    net_predictions.append(sum(np.where(y-n_predictions == 0,1,0))/len(n_predictions))

    log_model.fit(X_train,y)
    l_predictions = log_model.predict(X_test)
    log_predictions.append(sum(np.where(y-l_predictions == 0,1,0))/len(l_predictions))

4000


C:\Users\jonah\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


4000


C:\Users\jonah\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


4000


...

4000


C:\Users\jonah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\jonah\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


4000


C:\Users\jonah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


KeyboardInterrupt: 

In [16]:
splits = [5000,6000,7000,8000,9000,10000,11000,12000,13000]

df1 = (pd.DataFrame({'value': tree_predictions,'split':splits,'Model': 'Decision Tree'}))
df2 = (pd.DataFrame({'value': knn_predictions,'split':splits,'Model': 'KNN'}))
df3 = (pd.DataFrame({'value': net_predictions,'split':splits,'Model': 'Neural Net'}))
df4 = (pd.DataFrame({'value': log_predictions,'split':splits,'Model': 'Multinomial Regression'}))

ValueError: All arrays must be of the same length

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

df_plot = pd.concat([df1,df2,df3,df4])
sns.lmplot(data = df_plot,x='split',y='value',hue='Model', palette='coolwarm')
plt.xlabel('Number of authors')
plt.ylabel('Accuracy')

## Scaler vs No Scaler

In [ ]:
tree_predictions = []
no_scaler_t = []

for j in range (0,len(trains)):
    print(i)
    X_train = trains[j]
    X_test = tests[j]
    y = ys[j]

    tree_model = DecisionTreeClassifier(max_depth = 200)
    tree_model.fit(X_train,y)
    t_predictions = tree_model.predict(X_test)
    no_scaler_t.append(sum(np.where(y-t_predictions == 0,1,0))/len(t_predictions))

    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)

    tree_model = DecisionTreeClassifier(max_depth = 200)

    tree_model.fit(X_train,y)
    t_predictions = tree_model.predict(X_test)
    tree_predictions.append(sum(np.where(y-t_predictions == 0,1,0))/len(t_predictions))